In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')

In [20]:
from get_data import load_history, load_current, combine_history, process_pca, process_y, load_bios, load_team_data
# load_history()
# load_current()
# combine_history()
# load_team_data()
# load_bios()
process_y()

Processed y, writing to csv
Processed y, writing to csv


In [4]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LassoCV, LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import cross_val_score

def forward_feature_selection_with_elimination(X, y, col, tol=1e-6, cv=3):
    n_features = X.shape[1]
    selected_features = []
    remaining_features = X.columns.tolist()
    best_score = 0
    lasso_cv = LinearRegression()
    
    while remaining_features:
        scores = []
        n = len(remaining_features)
        for i, feature in enumerate(remaining_features):
            # Try adding the current feature to the selected features
            features_to_try = selected_features + [feature]
            X_subset = X[features_to_try]

            # Use cross-validation to evaluate the model
            score = np.mean(cross_val_score(lasso_cv, X_subset, y[col], cv=cv))
            scores.append((score, feature))
            if i % 100 == 0:
                print(i, '/', n)

        # Sort features by score
        scores.sort(key=lambda x: x[0], reverse=True)

        best_scores = [s for s in scores if s[0] - best_score > tol]
        
        # If the score improves, add the best feature
        if len(best_scores) > 0:
            selected_features.append(best_scores[0][1])
            remaining_features = [f for _, f in best_scores[1:]]  # Keep top n_to_keep - 1 remaining features
            best_score = best_scores[0][0]
            print(f"Selected feature {best_scores[0][1]} with score {best_score}")
        else:
            # Stop if no improvement
            break
        
    return selected_features, best_score

def run_fselection(X, y):
    features_map = {}
    for col in y.columns:
        print(f'Fitting for {col}')
        features_map[col] = forward_feature_selection_with_elimination(X, y, col)
    return feature_map

In [30]:
from process_data import get_player_stats, get_full_stats, PRED_COLS, run_training

In [6]:
import json

def load_player_feature_map(player_type):
    player_feature_map_file = f'data/{player_type}_player_features.json'
    try:
        with open(player_feature_map_file, 'r') as f:
            player_features_map = json.loads(f.read())
    except FileNotFoundError:
        print('File not found. Fitting...')
        X_p, y = get_player_stats(player_type)
        print('Data loaded')
        player_features_map = {}
        for col in y.columns:
            print(f'Fitting for {col}')
            player_features_map[col] = forward_feature_selection_with_elimination(X_p, y, col)
        with open(player_feature_map_file, 'w') as f:
            f.write(json.dumps(player_features_map))
    return player_features_map


def load_feature_maps(player_type='skater'):
    player_features_map = load_player_feature_map(player_type)
    
    feature_map_file = f'data/full_{player_type}_features.json'
    try:
        with open(feature_map_file, 'r') as f:
            full_features_map = json.loads(f.read())
    except FileNotFoundError:
        print('File not found. Fitting...')
        full_features_map = {}
        
        for col in PRED_COLS[player_type]:
            print(f'Fitting for {col}')
            X_p, y = get_full_stats(player_type, cols=player_features_map[col][0])       
            print('Loaded data')
            full_features_map[col] = forward_feature_selection_with_elimination(X_p, y, col)
        with open(feature_map_file, 'w') as f:
            f.write(json.dumps(full_features_map))
            
    return full_features_map, player_features_map




In [37]:
full_features_map, player_features_map = load_feature_maps()
pipes = run_training('skater', full_features_map, player_features_map, retrain=True)

g -- 0.08215589819597358
g -- 0.2691759409542468
a -- 0.0928994577565353
a -- 0.40650624968642257
sog -- 0.23158330461124488
sog -- 1.06760539188018
fow -- 0.7523712678711754
fow -- 0.7787308107006455
hit -- 0.2683402959400334
hit -- 0.9725752470849486
block -- 0.23555499964492932
block -- 0.7609384142545998
pim -- 0.02597477820674554
pim -- 0.706147428666212
goalsfor -- 0.05142459596541038
goalsfor -- 0.6330227565045711
goalsaga -- 0.03328360085634807
goalsaga -- 0.6266593657093465
ppp -- 0.09983279086078412
ppp -- 0.15502061060893887


In [38]:
goalie_features_map, g_player_features_map = load_feature_maps('goalie')
g_pipes = run_training('goalie', goalie_features_map, g_player_features_map, retrain=True)

ga -- 0.13166408400883123
ga -- 1.1629838282921716
win -- 0.07687043104624058
win -- 0.2578682460754789
so -- 0.18294616861468516
so -- 0.3725868408276006
save -- 0.17484469063985064
save -- 9.844253562557185
icetime -- 0.19608094919618224
icetime -- 1291.8171405179978


/opt/conda/envs/yh/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.908e+10, tolerance: 4.822e+06
  model = cd_fast.enet_coordinate_descent(
/opt/conda/envs/yh/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.240e+10, tolerance: 5.646e+06
  model = cd_fast.enet_coordinate_descent(


In [41]:
import pickle
pickle.dump(pipes, open('data/skater_models.p', 'wb'))
pickle.dump(g_pipes, open('data/goalie_models.p', 'wb'))